## 지진관측기록 시각화(Heat map)

In [ ]:
'''
xlsx file to 
'''
import jpype
import asposecells
jpype.startJVM()
from asposecells.api import Workbook, SaveFormat

workbook = Workbook("xlsx_file\\historical_earthquake.xlsx")
workbook.save("csv_file\\historical_earthquake.csv", SaveFormat.CSV)


workbook = Workbook("xlsx_file\\국내지진목록_1904-03-03_2023-05-25.xls")
workbook.save("csv_file\\국내지진목록_1904-03-03_2023-05-25.csv", SaveFormat.CSV)


In [ ]:
'''
2 ~ 1904 Earthquake
가중치 : 진도 -> 가속도(PGA)
'''
import pandas as pd
pd.set_option('mode.chained_assignment',  None)

Rom = {'Ⅰ' : 0.035, 'Ⅱ' : 0.15, 'Ⅲ' : 0.495, 'Ⅳ' : 1.66, 'Ⅴ' : 4.71, 'Ⅵ' : 10.795, 'Ⅶ' : 23.195, 'Ⅷ' : 49.835, 'Ⅸ' : 107.075, 'Ⅹ' : 146.14, '-' : 0, '?' : 0}     # 진도 rom num : arabic num


df_his = pd.read_csv("csv_file\\historical_earthquake.csv", encoding='utf-8')
df_his = df_his.rename(columns= df_his.iloc[0])
df_his = df_his.drop(df_his.index[0])
df_his = df_his.drop(df_his.index[len(df_his['진도']) - 1])             # column, 마지막 값(없는 값) 정리

df_his['일자(양력/음력)'] = df_his['일자(양력/음력)'].astype('str')         # 양력만 추출
df_his_date = df_his['일자(양력/음력)'].str.split('/')
df_his['일자'] = df_his_date.str.get(0)

j = 1                                                                 # 진도 arabic num 변환
for i in df_his['진도']:
    if len(i) == 1:
        temp = Rom[i]
        df_his['진도'][j] = temp
    elif len(i) == 3:
        temp = i[2]
        temp = Rom[temp]
        df_his['진도'][j] = temp

    j += 1

df_his['위도/경도'] = df_his['위도/경도'].astype('str')                     # 위도 , 경도 따로 추출, 실수화
df_his = df_his[df_his['위도/경도'] != '진앙미정 / 진앙미정']
df_his_loc = df_his['위도/경도'].str.split('/')
df_his['위도'] = df_his_loc.str.get(0)
df_his['경도'] = df_his_loc.str.get(1)

df_his = df_his.drop(['일자(양력/음력)', '위도/경도', '기사(출전)', '번호', '감진반경(km)'], axis= 'columns')       # column 최종 정리

import folium
from folium import plugins
latitude, longitude = 35.9078, 127.7669                                 # 대한민국의 좌표
S_korea = folium.Map(location = [latitude, longitude], zoom_start = 7)
df_his['위도'] = df_his['위도'].astype('float')
df_his['경도'] = df_his['경도'].astype('float')
df_his['진도'] = df_his['진도'].astype('float')
S_korea.add_child(plugins.HeatMap(zip(df_his['위도'], 
                                      df_his['경도'],
                                    df_his['진도']), radius= 15))
   

In [ ]:
'''
1904 03 03 ~ 2023 05 25 Earthquake
가중치 : 규모 -> Energy
'''
import pandas as pd
df_rec = pd.read_csv("csv_file\\국내지진목록_1904-03-03_2023-05-25.csv", encoding= 'utf-8')
df_rec = df_rec.rename(columns= df_rec.iloc[0])
df_rec = df_rec.drop(['번호', '발생시각', '위치', '지도보기', '상세정보\n(진도 등)', '최대\n진도', '깊이(km)'], axis= 'columns')
df_rec = df_rec.drop(df_rec.index[0:1])
df_rec = df_rec.dropna(axis=0)



df_rec['위도'] = df_rec['위도'].astype('str')
rec_lat = df_rec['위도'].str.split(' ')
df_rec['위도'] = rec_lat.str.get(0)

df_rec['경도'] = df_rec['경도'].astype('str')
rec_lat = df_rec['경도'].str.split(' ')
df_rec['경도'] = rec_lat.str.get(0)

j = 1
for i in df_rec['규모']:                        # 규모 -> 에너지 환산 :  https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=gumdansan21&logNo=130135121090
    temp = (11.8 + 1.5 * float(i))
    df_rec['규모'][j] = 10 ** temp
    j += 1
    
df_rec = df_rec[df_rec['위도'] != '-']


import folium                                                            # heatmap 
from folium import plugins
latitude, longitude = 35.9078, 127.7669                                 # 대한민국의 좌표
S_korea = folium.Map(location = [latitude, longitude], zoom_start = 7)

df_rec['위도'] = df_rec['위도'].astype('float')
df_rec['경도'] = df_rec['경도'].astype('float')

S_korea.add_child(plugins.HeatMap(zip(df_rec['위도'],
                                      df_rec['경도'],
                                      df_rec['규모']), radius= 15))






## 상관관계 분석 -> 지반 경도 : 진도

In [4]:
import folium
import json

city_data = {

    'Seoul': 0.20,
    'Busan': 0.12,
    'Daegu': 0.155,
    'Incheon': 0.201,
    'Gwangju': 0.183,
    'Daejeon': 0.196,
    'Ulsan': 0.214,
    'Gyeonggi-do': 0.247,
    'Gangwon-do': 0.124,
    'Chungcheongbuk-do': 0.142,
    'Chungcheongnam-do': 0.144,
    'Jeollabuk-do': 0.133,
    'Jeollanam-do': 0.102,
    'Gyeongsangbuk-do': 0.113,
    'Gyeongsangnam-do': 0.121,
    'Jeju': 0.174
}


geojson_file = "json_file\\skorea-provinces-geo.json"


with open(geojson_file, 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)


center = [36.5, 127.5]

folium 
map = folium.Map(location=center, zoom_start=7)



all_provinces = []  # 모든 지역을 저장하기 위한 리스트

for feature in geojson_data['features']:
    properties = feature['properties']
    province = properties['NAME_1']
    all_provinces.append(province)  # 모든 지역을 리스트에 추가
    
    if province in city_data:
        rate = city_data[province]
        color = 'blue'
        
        if rate <= 0.1:
            color = 'red'
        elif rate <= 0.15:
            color = 'orange'
        elif rate <= 0.2:
            color = 'yellow'
       
        folium.GeoJson(
            feature,
            style_function=lambda x, color=color: {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.7
            }
        ).add_to(map)

'''
# 모든 지역 출력
for province in all_provinces:
    print(province)
'''
# 지도를 HTML 파일로 저장
map.save('earthquake_map.html')



# 내진율에 따라 구역별로 색상 설정
for feature in geojson_data['features']:
    properties = feature['properties']
    province = properties['NAME_1']
 
    # 내진율 데이터가 있는 경우에만 색상 설정
    if province in city_data:
        rate = city_data[province]
        if rate <= 0.1:
            color = 'red'
        elif rate <= 0.15:
            color = 'orange'
        elif rate <= 0.2:
            color = 'yellow'
        else:
            color = 'green'

        folium.GeoJson(
            feature,
            style_function=lambda x: {
                'fillColor': color,
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0.7
            }
        ).add_to(map)